In [15]:
import pytimeloop.timeloopfe.v4 as tl
import time
import csv

NUM_THREADS = 1

spec = tl.Specification.from_yaml_files(
    "configs/timeloop_gemm_tpu.yaml",
    "ert/tpu_like.yaml"
)
spec.mapper.num_threads = NUM_THREADS

SEARCH_SIZE = [1000, 2000, 3000, 4000]

with open('timeloop_gemm_16k.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['search_size', 'threads', 'mapper_time', 'energy', 'latency'])
    for search_size in SEARCH_SIZE:
        start = time.time() 
        spec.mapper.search_size = search_size
        output = tl.call_mapper(spec, output_dir="outputs/timeloop")
        end = time.time()
        mapper_time = end - start
        writer.writerow([search_size, NUM_THREADS, mapper_time, output.energy, output.latency])

RuntimeError: 

========================================================================================================================
Timeloop mapper failed with return code -2. Please check the output files in outputs/timeloop for more information. To debug, you can edit the file:
	outputs/timeloop/parsed-processed-input.yaml
and run 
	tl mapper outputs/timeloop/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.

In [18]:
spec.architecture.nodes.find('LocalBuffer').constraints.dataspace = {'bypass': ['weight'], 'keep': ['input', 'output']}
spec.architecture.nodes.find('LocalBuffer').constraints.spatial.factors = ['C=1', 'Q=1', 'M=4']